 The optimization package of Qiskit is mainly for QUBO, or general quadratic problems. However, as we are dealing with max-4-cut, we have terms upto order 4. It seems that we need to implement the QAOA algorithm on a circuit level, unless there are other packages unknown to us.

Please put reusable functions into python files and keep the minimum amount of code in this notebook. <br>
Thanks to Jakob and Franziska, now we have a sample data to work on with. I (Kevin) suggest that we follow a similar structure as 'https://qiskit.org/textbook/ch-applications/qaoa.html'. 
1. Use brute force to compute the costs for all combinations and find the optimal cost. This will be used for   scoring the performance of our QAOA implementation.
2. Construct the mixing hamiltonian and problem Hamiltonian gate by gate and draw out the circuit diagrams. (For this refer to Jezer's document)
3. Assemble the circuit and draw the circuit diagram
4. Run classical optimization. (I think Qiskit has a few built-in classical optimizers that we can use.)
5. Evaluate the results, and check the performance for different problem sizes and circuit depth.

In [ ]:
from data_processing import *
from Max_k_cut_functions import *
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
from scipy.optimize import minimize

In [ ]:
# Set parameters here

p = 1  # the depth of QAOA

l = 2  # the number of qubits for each node, k = 2^l where k is the number of subgraphs we'd like to cut, 
       # or to say, the k in 'max-k-cut'. l = 2 for a max-4-cut example 
    
n_imp = 3  # the number of important nodes, or to say the nodes with which the associated edges have larger weights

n_unimp = 2  # the number of less important nodes ... smaller weights

nshots = 512  # the number of shots per iteration

init_params = [np.pi/8, np.pi]*p  # the initial parameters gammas and betas. This is however unused in the full
                                  # optimization loop since the educated global guess optimization does not require
                                  # initial guesses. The init_params here will be used later to demonstrate the 
                                  # performance of a local optimzation alone.

local_optimization_method = 'Nelder-Mead'  # the local optimization algorithm to be used, supported by Scipy. 
                                      # good candidates include 'Powell', 'COBYLA', 'Nelder-Mead', 'BFGS'
    
###################################################################################################################

k = 2**l  # see above
n = n_imp + n_unimp  # total number of nodes

In [ ]:
# Generate sample data

G = generate_data(n_unimp, n_imp)
pos = nx.spring_layout(G)
nx.draw(G, pos, with_labels=True)
nx.draw_networkx_edge_labels(G, pos)
plt.show()

In [ ]:
# Get a matrix representation of the problem and use Brute force to obtain the optimal solution

weights = get_weight_matrix(G)  # Obtain the weight matrix

R = compute_normalization_scale(weights)  # The normalization scale suggested by the paper

rescaled_weights = weights / R  # The rescaled weight matrix

cost_of_graph, partition = brut_force(G,k)  # Uses brute force to find the optimal solution, so that we have a reference 
                                   # to test our algorithm
    
normalized_cost = cost_of_graph / R


print('original weight matrix =', weights)
print('normalization factor =', R)
print('rescaled weight matrix =', rescaled_weights)
print('original cost =', cost_of_graph)
print('normalized cost =', normalized_cost)

In [ ]:
param_history, cost_history, circ_history = full_optimization_loop(n, l, rescaled_weights, p, 
                                                                   local_optimization_method=local_optimization_method, 
                                                                   optimal_cost=normalized_cost)

In [ ]:
param_history

In [ ]:
cost_history

In [ ]:
#circ_history[-1].decompose().draw()

In [ ]:
circ = make_full_circuit(n,l,rescaled_weights,p)  # building the full circuit
counts, transpiled_circ = run_circuit(circ, param_history[-1], nshots=nshots)  # do a final run with optimal parameters found
show_distribution(counts, l)

## Algorithm broke into parts

### Build the circuit

In [ ]:
mb = make_mixing_block(n,l,1)  # building the mixing block
#mb.draw()

In [ ]:
cb = make_cost_block(n,l,rescaled_weights,1)  # building the cost block 
#cb.draw()

In [ ]:
ib = make_initial_block(n,l)  # building the initialization block
#ib.draw()

In [ ]:
circ = make_full_circuit(n,l,rescaled_weights,p)  # building the full circuit
#circ.decompose().draw()

### Run a sample circuit

In [ ]:
counts, transpiled_circ = run_circuit(circ, init_params, nshots=nshots)  # visualize the circuit to be runned
transpiled_circ.decompose().draw()

In [ ]:
average_cost = compute_cost(counts, l, rescaled_weights, n_counts = nshots)  # compute the cost from the sample run 
                                                                             # above
average_cost

### Run QAOA with only a local optimization algorithm 

In [ ]:
func_to_optimize = func_to_optimize_wrapper(circ, l, rescaled_weights, nshots=512, simulator='aer_simulator')

In [ ]:
res = minimize(func_to_optimize, init_params, method=local_optimization_method)
print('cost', -1*res.fun)
print('approximation_ratio', -1*res.fun / normalized_cost)
print('parameters', res.x)

# Benchmarking

In [ ]:
distribution_qaoa = show_distribution(counts, l)

In [ ]:
counts_accept=nshots*0.05
len_d = len(distribution_qaoa.values())

In [ ]:
Prob_distribution = dict()
for key, value in distribution_qaoa.items():
    if value > counts_accept:
        Prob_distribution[key]=value

In [ ]:
import ast

In [ ]:
for key, value in Prob_distribution.items():
    P = {}
    participation_data = ast.literal_eval(key)  
    for i in range(len(participation_data)):
        P["P"+str(i)] = participation_data[i]
    a,b = cost(G, P)

# Key muss array sein, kein string

In [ ]:

print(partition)